# Epistolary

This tool is designed to "print" emails to a PDF file (one thread per file), with a blank (ruled) page after each email. 
You can write a reply to the email on the blank page, and Epistolary will convert your handwriting to text and send it as a reply to the email.

It is originally designed to be used with the [Remarkable](https://remarkable.com/) tablet, which is a great device for reading and annotating PDFs, but it should work with standalone PDFs, tablet devices, or scanned documents as well.

## Architecture

The tool comprises three main components:

* `MailboxManager`: A class that manages the mailbox, and provides methods to get the next email to be printed, and to send a reply to an email.
* `DocumentManager`: A class that manages the PDF document library.
* `EpistolaryOrchestrator`: A class that manages the interaction between the `MailboxManager` and the `DocumentManager`, and provides OCR and main entry point functionality.



In [2]:
from epistolary import MailboxManager, DocumentManager, Document, DocumentID, EmailID, EpistolaryOrchestrator

In [234]:
from redbox import EmailBox
from redbox.models import EmailMessage
from redbox.query import UID
from redmail import EmailSender

from epistolary import EmailID


def _get_msgid_from_header_dict(header_dict: dict[str, str]) -> EmailID:
    _possible_message_id_keys = [
        "Message-ID",
        "Message-Id",
        "Message-id",
        "message-id",
        "message_id",
        "messageid",
    ]
    for possible_key in _possible_message_id_keys:
        if possible_key in header_dict:
            return EmailID(header_dict[possible_key])

    raise ValueError("No message ID found in header dict")


class SMTPIMAPMailboxManager(MailboxManager):
    """
    A class representing a mailbox manager for SMTP and IMAP protocols.

    Args:
        host (str): The hostname of the mail server.
        port (int): The port number of the mail server.
        username (str): The username for authentication.
        password (str): The password for authentication.
    """

    def __init__(
        self,
        imap_host: str,
        imap_port: int,
        username: str,
        password: str,
        smtp_host: str,
        smtp_port: int = 465,
        smtp_username: str | None = None,
        smtp_password: str | None = None,
    ):
        """Create a new SMTPIMAPMailboxManager object.

        Arguments:
            host (str): The hostname of the mail server.
            port (int): The port number of the mail server.
            username (str): The username for authentication.
            password (str): The password for authentication.

        """
        if smtp_username is None:
            smtp_username = username
        if smtp_password is None:
            smtp_password = password
        self._box = EmailBox(imap_host, imap_port, username, password)
        self._sender = EmailSender(smtp_host, smtp_port, smtp_username, smtp_password)

    def get_emails(self, folder: str | None = None) -> dict[EmailID, EmailMessage]:
        """
        Get the emails from the specified folder.

        Arguments:
            folder (str | None, optional): The folder name. Defaults to "INBOX".

        Returns:
            dict[EmailID, EmailMessage]: A dictionary containing the emails,
                where keys are the email IDs and the values are the messages.
        """
        if folder is None:
            folder = "INBOX"
        messages = self._box[folder].search(unseen=True)
        return {
            EmailID(_get_msgid_from_header_dict(message.headers)): message
            for message in messages
        }

    def get_email(self, email_id: EmailID) -> EmailMessage:
        """
        Get the email with the specified ID.

        Arguments:
            email_id (EmailID): The email ID.

        Returns:
            EmailMessage: The email message.
        """
        # TODO: Sad puppy
        return self.get_emails()[email_id]

    def get_email_subject_and_text(self, uid: EmailID) -> tuple[str, str]:
        """
        Get the subject and text of the email with the specified ID.

        Arguments:
            uid (EmailID): The email ID.

        Returns:
            tuple[str, str]: The subject and text of the email.
        """
        email = self.get_email(uid)
        return email.subject, email.html_body

    def send_message(
        self, to: str, subject: str, body: str, in_reply_to: EmailID | None = None
    ) -> bool:
        """Send a message."""
        to = "epistolary-testing@matelsky.com"
        # sender = self._sender.username
        # sender = "jordan@matelsky.com"
        self._sender.send(
            subject=subject,
            receivers=[to],
            text=body,
            html=body,
            headers={"In-Reply-To": in_reply_to} if in_reply_to is not None else None,
        )
        return True

In [235]:
# import getpass

# pw = getpass.getpass()

In [236]:
SM = SMTPIMAPMailboxManager(
    "imap.zoho.com", 993, "jordan@matelsky.com", pw, "smtp.zoho.com", 587
)

In [237]:
# SM.send_message(
#     "",
#     "Re: This is a test!",
#     "Using the Epistolary library AGAIN in the same thread."
# )

In [238]:
es = SM.get_emails()

In [239]:
es

{'<m_sN0KUBTZScLWiP2ZAMmQ@geopod-ismtpd-37>': EmailMessage(session=<imaplib.IMAP4_SSL object at 0x106c591d0>, uid=11, mailbox='INBOX'),
 '<20231128141217.33512585.429933@sailthru.com>': EmailMessage(session=<imaplib.IMAP4_SSL object at 0x106c591d0>, uid=13, mailbox='INBOX'),
 '<ktl0Qse0TtCq0h1mEB04oA@geopod-ismtpd-79>': EmailMessage(session=<imaplib.IMAP4_SSL object at 0x106c591d0>, uid=14, mailbox='INBOX'),
 '<SJ0PR11MB5920682780F7C7E026E7F569C383A@SJ0PR11MB5920.namprd11.prod.outlook.com>': EmailMessage(session=<imaplib.IMAP4_SSL object at 0x106c591d0>, uid=15, mailbox='INBOX'),
 '<8eacf65b13225dc16a354c7e3.591ef77aae.20231129155552.8c8e41e83d.e9816ece@mail72.atl51.rsgsv.net>': EmailMessage(session=<imaplib.IMAP4_SSL object at 0x106c591d0>, uid=18, mailbox='INBOX'),
 '<CAFR5dqSyfUTdNJJOw8V9E0Drr0N3bY_mGviTBA5AUQwX1+WRGg@mail.gmail.com>': EmailMessage(session=<imaplib.IMAP4_SSL object at 0x106c591d0>, uid=19, mailbox='INBOX'),
 '<20231129124417.33523803.430928@sailthru.com>': EmailMess